# Overview
- nb14ベース
- ハイパラチューニングする

# Const

In [19]:
NB = '019'
isSmallSet = False
if isSmallSet:
    LENGTH = 7000
else:
    LENGTH = 500_000

PATH_TRAIN = './../data/input/train_clean.csv'
PATH_TEST = './../data/input/test_clean.csv'
PATH_SMPLE_SUB = './../data/input/sample_submission.csv'
DIR_OUTPUT = './../data/output/'
cp = ['#f8b195', '#f67280', '#c06c84', '#6c5b7b', '#355c7d']
sr = 10*10**3  # 10 kHz

# Import everything I need :)

In [20]:
import warnings
warnings.filterwarnings('ignore')
import time
import gc
import itertools
import multiprocessing
import numpy as np
from scipy import signal
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from fastprogress import progress_bar
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold, train_test_split, StratifiedKFold
from sklearn.metrics import f1_score, mean_absolute_error, confusion_matrix
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Lasso
import optuna
# from dtreeviz.trees import dtreeviz

# My function

In [21]:
def f1_macro(true, pred):
    return f1_score(true, pred, average='macro')

def get_df_batch(df, batch):
    idxs = df['batch'] == batch
    assert any(idxs), 'そのようなbatchはありません'
    return df[idxs]
    
def get_signal_mv_mean(df, n=3001):
    signal_mv = np.zeros(len(df))
    for bt in df['batch'].unique():
        idxs = df['batch'] == bt
        _signal_mv = df['signal'][idxs].rolling(n, center=True).mean().interpolate('spline', order=5, limit_direction='both').values
        signal_mv[idxs] = _signal_mv
    return signal_mv

def get_signal_mv_std(df, n=3001):
    signal_mv = np.zeros(len(df))
    for bt in df['batch'].unique():
        idxs = df['batch'] == bt
        _signal_mv = df['signal'][idxs].rolling(n, center=True).std().interpolate('spline', order=5, limit_direction='both').values
        signal_mv[idxs] = _signal_mv
    return signal_mv

def get_signal_mv_min(df, n=3001):
    signal_mv = np.zeros(len(df))
    for bt in df['batch'].unique():
        idxs = df['batch'] == bt
        _signal_mv = df['signal'][idxs].rolling(n, center=True).min().interpolate('spline', order=5, limit_direction='both').values
        signal_mv[idxs] = _signal_mv
    return signal_mv

def get_signal_mv_max(df, n=3001):
    signal_mv = np.zeros(len(df))
    for bt in df['batch'].unique():
        idxs = df['batch'] == bt
        _signal_mv = df['signal'][idxs].rolling(n, center=True).max().interpolate('spline', order=5, limit_direction='both').values
        signal_mv[idxs] = _signal_mv
    return signal_mv

class permutation_importance():
    def __init__(self, model, metric):
        self.is_computed = False
        self.n_feat = 0
        self.base_score = 0
        self.model = model
        self.metric = metric
        self.df_result = []
    
    def compute(self, X_valid, y_valid):
        self.n_feat = len(X_valid.columns)
        if self.metric == 'auc':
            y_valid_score = self.model.predict_proba(X_valid)[:, 1]
            fpr, tpr, thresholds = roc_curve(y_valid, y_valid_score)
            self.base_score = auc(fpr, tpr)
        else:
            pred = np.round(self.model.predict(X_valid)).astype('int8')
            self.base_score = self.metric(y_valid, pred)
        self.df_result = pd.DataFrame({'feat': X_valid.columns, 
                                       'score': np.zeros(self.n_feat),
                                       'score_diff': np.zeros(self.n_feat)})
        
        # predict
        for i, col in enumerate(X_valid.columns):
            df_perm = X_valid.copy()
            np.random.seed(1)
            df_perm[col] = np.random.permutation(df_perm[col])
            y_valid_pred = self.model.predict(df_perm)
            if self.metric == 'auc':
                y_valid_score = self.model.predict_proba(df_perm)[:, 1]
                fpr, tpr, thresholds = roc_curve(y_valid, y_valid_score)
                score = auc(fpr, tpr)
            else:
                score = self.metric(y_valid, np.round(y_valid_pred).astype('int8'))
            self.df_result['score'][self.df_result['feat']==col] = score
            self.df_result['score_diff'][self.df_result['feat']==col] = self.base_score - score
        self.is_computed = True
    
    def get_negative_feature(self):
        assert self.is_computed!=False, 'compute メソッドが実行されていません'
        idx = self.df_result['score_diff'] < 0
        return self.df_result.loc[idx, 'feat'].values.tolist()
        
    def get_positive_feature(self):
        assert self.is_computed!=False, 'compute メソッドが実行されていません'
        idx = self.df_result['score_diff'] > 0
        return self.df_result.loc[idx, 'feat'].values.tolist()
    
    def show_permutation_importance(self, score_type='loss'):
        '''score_type = 'loss' or 'accuracy'  '''
        assert self.is_computed!=False, 'compute メソッドが実行されていません'
        if score_type=='loss':
            ascending = True
        elif score_type=='accuracy':
            ascending = False
        else:
            ascending = ''
        
        plt.figure(figsize=(15, int(0.25*self.n_feat)))
        sns.barplot(x="score_diff", y="feat", data=self.df_result.sort_values(by="score_diff", ascending=ascending))
        plt.title('base_score - permutation_score')

def plot_corr(df, abs_=False, threshold=0.95):
    if abs_==True:
        corr = df.corr().abs()>threshold
        vmin = 0
    else:
        corr = df.corr()
        vmin = -1

    # Plot
    fig, ax = plt.subplots(figsize=(12, 10), dpi=100)
    fig.patch.set_facecolor('white')
    sns.heatmap(corr,
                xticklabels=df.corr().columns,
                yticklabels=df.corr().columns,
                vmin=vmin,
                vmax=1,
                center=0, 
                annot=False)

    # Decorations
    ax.set_title('Correlation', fontsize=22)

def get_low_corr_column(df, threshold):

    df_corr = df.corr()
    df_corr = abs(df_corr)
    columns = df_corr.columns

    # 対角線の値を0にする
    for i in range(0, len(columns)):
        df_corr.iloc[i, i] = 0

    while True:
        columns = df_corr.columns
        max_corr = 0.0
        query_column = None
        target_column = None

        df_max_column_value = df_corr.max()
        max_corr = df_max_column_value.max()
        query_column = df_max_column_value.idxmax()
        target_column = df_corr[query_column].idxmax()

        if max_corr < threshold:
            # しきい値を超えるものがなかったため終了
            break
        else:
            # しきい値を超えるものがあった場合
            delete_column = None
            saved_column = None

            # その他との相関の絶対値が大きい方を除去
            if sum(df_corr[query_column]) <= sum(df_corr[target_column]):
                delete_column = target_column
                saved_column = query_column
            else:
                delete_column = query_column
                saved_column = target_column

            # 除去すべき特徴を相関行列から消す（行、列）
            df_corr.drop([delete_column], axis=0, inplace=True)
            df_corr.drop([delete_column], axis=1, inplace=True)

    return df_corr.columns  # 相関が高い特徴量を除いた名前リスト

In [22]:
def train_lgbm(X, y, X_te, lgbm_params, random_state=5, n_fold=5, verbose=50, early_stopping_rounds=100, show_fig=True):
    # using features
    print(f'features({len(X.columns)}): \n{X.columns}') if not verbose==0 else None

#     folds = KFold(n_splits=n_fold, shuffle=True, random_state=random_state)
    folds = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=random_state)

    scores = []
    oof = np.zeros(len(X))
    oof_round = np.zeros(len(X))
    test_pred = np.zeros(len(X_te))
    df_pi = pd.DataFrame(columns=['feat', 'score_diff'])
    for fold_n, (train_idx, valid_idx) in enumerate(folds.split(X, y=y)):
        if  verbose==0:
            pass
        else:
            print('\n------------------')
            print(f'- Fold {fold_n + 1}/{N_FOLD} started at {time.ctime()}')

        # prepare dataset
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_valid = y[train_idx], y[valid_idx]

        # train
        model = LGBMRegressor(**lgbm_params, n_estimators=N_ESTIMATORS)
        model.fit(X_train, y_train,
                  eval_set=[(X_train, y_train), (X_valid, y_valid)],
                  verbose=verbose,
                  early_stopping_rounds=early_stopping_rounds)

        # pred
        y_valid_pred = model.predict(X_valid, model.best_iteration_)
        y_valid_pred_round = np.round(y_valid_pred).astype('int8')
        _test_pred = model.predict(X_te, model.best_iteration_)

        # permutation importance
        pi = permutation_importance(model, f1_macro) # model と metric を渡す
        pi.compute(X_valid, y_valid)
        pi_result = pi.df_result

        # result
        oof[valid_idx] = y_valid_pred
        oof_round[valid_idx] = y_valid_pred_round
        score = f1_score(y_valid, y_valid_pred_round, average='macro')
        scores.append(score)
        test_pred += _test_pred
        df_pi = pd.concat([df_pi, pi_result[['feat', 'score_diff']]])
        if verbose==0:
            pass
        else:
            print(f'---> f1-score(macro) valid: {f1_score(y_valid, y_valid_pred_round, average="macro"):.4f}')
            print('')


    print('====== finish ======')
    print('score list:', scores)
    print('CV mean score(f1_macro): {0:.4f}, std: {1:.4f}'.format(np.mean(scores), np.std(scores)))
    print(f'oof score(f1_macro): {f1_score(y, oof_round, average="macro"):.4f}')
    print('')


    if show_fig==False:
        pass
    else:
        # visualization
        plt.figure(figsize=(5, 5))
        plt.plot([0, 10], [0, 10], color='gray')
        plt.scatter(y, oof, alpha=0.05, color=cp[1])
        plt.xlabel('true')
        plt.ylabel('pred')
        plt.show()
          
        # confusion_matrix
        plot_confusion_matrix(y, oof_round, classes=np.arange(11))
        
        
        # permutation importance
        plt.figure(figsize=(15, int(0.25*len(X.columns))))
        order = df_pi.groupby(["feat"]).mean()['score_diff'].reset_index().sort_values('score_diff', ascending=False)
        sns.barplot(x="score_diff", y="feat", data=df_pi, order=order['feat'])
        plt.title('base_score - permutation_score')
        plt.show()

    # submission
    test_pred = test_pred/N_FOLD
    test_pred_round = np.round(test_pred).astype('int8')
      
    return test_pred_round, test_pred, oof_round, oof

def plot_confusion_matrix(truth, pred, classes, normalize=False, title=''):
    cm = confusion_matrix(truth, pred)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    plt.figure(figsize=(10, 10))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion matrix', size=15)
    plt.colorbar(fraction=0.046, pad=0.04)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.grid(False)
    plt.tight_layout()

In [23]:
def train_test_split_lgbm(X, y, X_te, lgbm_params, random_state=5, test_size=0.3, verbose=50, early_stopping_rounds=100, show_fig=True):
    # using features
    print(f'features({len(X.columns)}): \n{X.columns}') if not verbose==0 else None

#     folds = KFold(n_splits=n_fold, shuffle=True, random_state=random_state)
#     folds = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=random_state)
    
    # prepare dataset
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=test_size, random_state=random_state)

    # train
    model = LGBMRegressor(**lgbm_params, n_estimators=N_ESTIMATORS)
    model.fit(X_train, y_train,
              eval_set=[(X_train, y_train), (X_valid, y_valid)],
              verbose=verbose,
              early_stopping_rounds=early_stopping_rounds)

    # pred
    oof = model.predict(X_valid, model.best_iteration_)
    oof_round = np.round(oof).astype('int8')
    test_pred = model.predict(X_te, model.best_iteration_)
    test_pred_round = np.round(test_pred).astype('int8')

    print('====== finish ======')
    print(f'oof score(f1_macro): {f1_score(y_valid, oof_round, average="macro"):.4f}')
    print('')


    if show_fig==False:
        pass
    else:
        # visualization
        plt.figure(figsize=(5, 5))
        plt.plot([0, 10], [0, 10], color='gray')
        plt.scatter(y_valid, oof, alpha=0.05, color=cp[1])
        plt.xlabel('true')
        plt.ylabel('pred')
        plt.show()
          
        # confusion_matrix
        plot_confusion_matrix(y_valid, oof_round, classes=np.arange(11))
        
        # permutation importance
        pi = permutation_importance(model, f1_macro) # model と metric を渡す
        pi.compute(X_valid, y_valid)
        pi.show_permutation_importance(score_type='accuracy')  # loss or accuracy
        plt.show()

    return test_pred_round, test_pred, oof_round, oof

<br>

ref: https://www.kaggle.com/martxelo/fe-and-ensemble-mlp-and-lgbm

In [24]:
def calc_gradients(s, n_grads=4):
    '''
    Calculate gradients for a pandas series. Returns the same number of samples
    '''
    grads = pd.DataFrame()
    
    g = s.values
    for i in range(n_grads):
        g = np.gradient(g)
        grads['grad_' + str(i+1)] = g
        
    return grads


def calc_low_pass(s, n_filts=10):
    '''
    Applies low pass filters to the signal. Left delayed and no delayed
    '''
#     wns = np.logspace(-2, -0.3, n_filts)
    wns = [0.3244]
    
    low_pass = pd.DataFrame()
    x = s.values
    for wn in wns:
        b, a = signal.butter(1, Wn=wn, btype='low')
        zi = signal.lfilter_zi(b, a)
        low_pass['lowpass_lf_' + str('%.4f' %wn)] = signal.lfilter(b, a, x, zi=zi*x[0])[0]
        low_pass['lowpass_ff_' + str('%.4f' %wn)] = signal.filtfilt(b, a, x)
        
    return low_pass

def calc_high_pass(s, n_filts=10):
    '''
    Applies high pass filters to the signal. Left delayed and no delayed
    '''
#     wns = np.logspace(-2, -0.1, n_filts)
    wns = [0.0100, 0.0264, 0.0699, 0.3005, 0.4885, 0.7943]
    
    high_pass = pd.DataFrame()
    x = s.values
    for wn in wns:
        b, a = signal.butter(1, Wn=wn, btype='high')
        zi = signal.lfilter_zi(b, a)
        high_pass['highpass_lf_' + str('%.4f' %wn)] = signal.lfilter(b, a, x, zi=zi*x[0])[0]
        high_pass['highpass_ff_' + str('%.4f' %wn)] = signal.filtfilt(b, a, x)
        
    return high_pass

def calc_roll_stats(s, windows=[10, 50, 100, 500, 1000, 3000]):
    '''
    Calculates rolling stats like mean, std, min, max...
    '''
    roll_stats = pd.DataFrame()
    for w in windows:
#         roll_stats['roll_mean_' + str(w)] = s.rolling(window=w, min_periods=1).mean().interpolate('spline', order=5, limit_direction='both')
        roll_stats['roll_std_' + str(w)] = s.rolling(window=w, min_periods=1).std().interpolate('spline', order=5, limit_direction='both')
        roll_stats['roll_min_' + str(w)] = s.rolling(window=w, min_periods=1).min().interpolate('spline', order=5, limit_direction='both')
        roll_stats['roll_max_' + str(w)] = s.rolling(window=w, min_periods=1).max().interpolate('spline', order=5, limit_direction='both')
        roll_stats['roll_range_' + str(w)] = roll_stats['roll_max_' + str(w)] - roll_stats['roll_min_' + str(w)]
        roll_stats['roll_q10_' + str(w)] = s.rolling(window=w, min_periods=1).quantile(0.10).interpolate('spline', order=5, limit_direction='both')
#         roll_stats['roll_q25_' + str(w)] = s.rolling(window=w, min_periods=1).quantile(0.25).interpolate('spline', order=5, limit_direction='both')
        roll_stats['roll_q50_' + str(w)] = s.rolling(window=w, min_periods=1).quantile(0.50).interpolate('spline', order=5, limit_direction='both')
#         roll_stats['roll_q75_' + str(w)] = s.rolling(window=w, min_periods=1).quantile(0.75).interpolate('spline', order=5, limit_direction='both')
#         roll_stats['roll_q90_' + str(w)] = s.rolling(window=w, min_periods=1).quantile(0.90).interpolate('spline', order=5, limit_direction='both')
    
    # add zeros when na values (std)
#     roll_stats = roll_stats.fillna(value=0)
             
    return roll_stats

# def calc_ewm(s, windows=[10, 50, 100, 500, 1000, 3000]):
#     '''
#     Calculates exponential weighted functions
#     '''
#     ewm = pd.DataFrame()
#     for w in windows:
#         ewm['ewm_mean_' + str(w)] = s.ewm(span=w, min_periods=1).mean()
#         ewm['ewm_std_' + str(w)] = s.ewm(span=w, min_periods=1).std()
        
#     # add zeros when na values (std)
#     ewm = ewm.fillna(value=0)
        
#     return ewm



def divide_and_add_features(s, signal_size=500000):
    '''
    Divide the signal in bags of "signal_size".
    Normalize the data dividing it by 15.0
    '''
    # normalize
    s = s/15.0
    
    ls = []
    for i in progress_bar(range(int(s.shape[0]/signal_size))):
        sig = s[i*signal_size:(i+1)*signal_size].copy().reset_index(drop=True)
        sig_featured = add_features(sig)
        ls.append(sig_featured)
    
    return pd.concat(ls, axis=0)

# Preparation

setting

In [25]:
sns.set()

<br>

load dataset

In [26]:
df_tr = pd.read_csv(PATH_TRAIN)
df_te = pd.read_csv(PATH_TEST)

<br>

処理のしやすさのために、バッチ番号を振る

In [27]:
batch_list = []
for n in range(10):
    batchs = np.ones(500000)*n
    batch_list.append(batchs.astype(int))
batch_list = np.hstack(batch_list)
df_tr['batch'] = batch_list

batch_list = []
for n in range(4):
    batchs = np.ones(500000)*n
    batch_list.append(batchs.astype(int))
batch_list = np.hstack(batch_list)
df_te['batch'] = batch_list

<br>

smallset?

In [28]:
if isSmallSet:
    print('small set mode')
    # train
    batchs = df_tr['batch'].values
    dfs = []
    for i_bt, bt in enumerate(df_tr['batch'].unique()):
        idxs = batchs == bt
        _df = df_tr[idxs][:LENGTH].copy()
        dfs.append(_df)
    df_tr = pd.concat(dfs).reset_index(drop=True)
    
    # test
    batchs = df_te['batch'].values
    dfs = []
    for i_bt, bt in enumerate(df_te['batch'].unique()):
        idxs = batchs == bt
        _df = df_te[idxs][:LENGTH].copy()
        dfs.append(_df)
    df_te = pd.concat(dfs).reset_index(drop=True)

# Train

In [29]:
def add_features(s):
    '''
    All calculations together
    '''
    feat_list = [s]
    feat_list.append(calc_gradients(s))
    feat_list.append(calc_low_pass(s))
    feat_list.append(calc_high_pass(s))
    feat_list.append(calc_roll_stats(s))
#     feat_list.append(calc_ewm(s))
    
    return pd.concat(feat_list, axis=1)

In [30]:
X = divide_and_add_features(df_tr['signal'], signal_size=LENGTH)
_feats = get_low_corr_column(X, threshold=0.97).to_list()
_feats.append('signal')
X = X[_feats]

X_te = divide_and_add_features(df_te['signal'], signal_size=LENGTH)
X_te = X_te[_feats]

y = df_tr['open_channels'].values

# hyper params optimization
- oputuna をつかう

In [43]:
# 最小化したい関数
N_ESTIMATORS = 500          # 最大学習回数
# N_ESTIMATORS = 20          # 最大学習回数
VERBOSE = 100               # 300回ごとに評価する
EARLY_STOPPING_ROUNDS = 50  # 200回の学習でよくならなければ、学習をとめる
# N_JOBS = multiprocessing.cpu_count() - 2
N_JOBS = 18
def objective(trial):
    print('')
    lgbm_params = {
              'max_depth': trial.suggest_int('max_depth', 1, 200),
              'min_child_samples': trial.suggest_int('min_chiled_samples', 5, 500),
              'min_child_weight': trial.suggest_int('min_chiled_weight', 1, 12),
              'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.5),
              'colsample_bytree': trial.suggest_discrete_uniform('colsample_bytree', 0.7, 1, 0.1),
              'subsample': trial.suggest_discrete_uniform('sugsample', 0.7, 1, 0.1),
              'lambda_l1' : trial.suggest_uniform('lambda_l1' , 0.0, 3.0),
              'lambda_l2' : trial.suggest_uniform('lambda_l2' , 0.0, 3.0),
             }

    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.33, random_state=42, stratify=y)
    
    model = LGBMRegressor(**lgbm_params, n_estimators=N_ESTIMATORS)
    model.fit(X_train, y_train,
              eval_set=[(X_train, y_train), (X_valid, y_valid)],
              verbose=0,
              early_stopping_rounds=EARLY_STOPPING_ROUNDS
             )

    
    
    y_valid_pred = model.predict(X_valid, model.best_iteration_)
    y_valid_pred_round = np.round(y_valid_pred).astype('int8')
    score = f1_score(y_valid, y_valid_pred_round, average="macro")
     
    return -score

In [44]:
%%time
# パラメータ探索
study = optuna.create_study()
study.optimize(objective, n_trials=300)

[I 2020-04-29 09:17:13,365] Finished trial#0 resulted in value: -0.935725785040066. Current best value is -0.935725785040066 with parameters: {'max_depth': 32, 'min_chiled_samples': 82, 'min_chiled_weight': 7, 'learning_rate': 0.12156822713235231, 'colsample_bytree': 1.0, 'sugsample': 1.0, 'lambda_l1': 1.0858089332938854, 'lambda_l2': 1.7229416631796926}.


[I 2020-04-29 09:19:07,336] Finished trial#1 resulted in value: -0.9355642235379474. Current best value is -0.935725785040066 with parameters: {'max_depth': 32, 'min_chiled_samples': 82, 'min_chiled_weight': 7, 'learning_rate': 0.12156822713235231, 'colsample_bytree': 1.0, 'sugsample': 1.0, 'lambda_l1': 1.0858089332938854, 'lambda_l2': 1.7229416631796926}.


[I 2020-04-29 09:21:20,316] Finished trial#2 resulted in value: -0.9311739308443563. Current best value is -0.935725785040066 with parameters: {'max_depth': 32, 'min_chiled_samples': 82, 'min_chiled_weight': 7, 'learning_rate': 0.12156822713235231, 'colsample_bytree': 1.0, 'sugsample': 1.0, 'lambda_l1': 1.0858089332938854, 'lambda_l2': 1.7229416631796926}.


[I 2020-04-29 09:23:47,340] Finished trial#3 resulted in value: -0.9317819868252488. Current best value is -0.935725785040066 with parameters: {'max_depth': 32, 'min_chiled_samples': 82, 'min_chiled_weight': 7, 'learning_rate': 0.12156822713235231, 'colsample_bytree': 1.0, 'sugsample': 1.0, 'lambda_l1': 1.0858089332938854, 'lambda_l2': 1.7229416631796926}.


[I 2020-04-29 09:25:34,793] Finished trial#4 resulted in value: -0.8573739689118676. Current best value is -0.935725785040066 with parameters: {'max_depth': 32, 'min_chiled_samples': 82, 'min_chiled_weight': 7, 'learning_rate': 0.12156822713235231, 'colsample_bytree': 1.0, 'sugsample': 1.0, 'lambda_l1': 1.0858089332938854, 'lambda_l2': 1.7229416631796926}.


...

[I 2020-04-29 13:32:19,445] Finished trial#159 resulted in value: -0.9356589799082499. Current best value is -0.9361077850947903 with parameters: {'max_depth': 187, 'min_chiled_samples': 147, 'min_chiled_weight': 6, 'learning_rate': 0.23713836611727865, 'colsample_bytree': 1.0, 'sugsample': 1.0, 'lambda_l1': 2.8943680775978597, 'lambda_l2': 1.3727573361886785}.


[I 2020-04-29 13:33:34,400] Finished trial#160 resulted in value: -0.9356735366032993. Current best value is -0.9361077850947903 with parameters: {'max_depth': 187, 'min_chiled_samples': 147, 'min_chiled_weight': 6, 'learning_rate': 0.23713836611727865, 'colsample_bytree': 1.0, 'sugsample': 1.0, 'lambda_l1': 2.8943680775978597, 'lambda_l2': 1.3727573361886785}.


[I 2020-04-29 13:35:21,812] Finished trial#161 resulted in value: -0.9358215374566682. Current best value is -0.9361077850947903 with parameters: {'max_depth': 187, 'min_chiled_samples': 147, 'min_chiled_weight': 6, 'learning_rate': 0.23713836611727865, 'colsample_bytree': 1.0, 'sugsample': 1.0, 'lambda_l1': 2.8943680775978597, 'lambda_l2': 1.3727573361886785}.


[I 2020-04-29 13:37:32,785] Finished trial#162 resulted in value: -0.9359315340011409. Current best value is -0.9361077850947903 with parameters: {'max_depth': 187, 'min_chiled_samples': 147, 'min_chiled_weight': 6, 'learning_rate': 0.23713836611727865, 'colsample_bytree': 1.0, 'sugsample': 1.0, 'lambda_l1': 2.8943680775978597, 'lambda_l2': 1.3727573361886785}.


KeyboardInterrupt: 

In [45]:
print(study.best_trial)
print(f'\n best_score = {study.best_value:.3f}')
print(f' best_params = {study.best_params}')

FrozenTrial(number=156, state=TrialState.COMPLETE, value=-0.9361077850947903, datetime_start=datetime.datetime(2020, 4, 29, 13, 25, 58, 691190), datetime_complete=datetime.datetime(2020, 4, 29, 13, 27, 40, 119116), params={'max_depth': 187, 'min_chiled_samples': 147, 'min_chiled_weight': 6, 'learning_rate': 0.23713836611727865, 'colsample_bytree': 1.0, 'sugsample': 1.0, 'lambda_l1': 2.8943680775978597, 'lambda_l2': 1.3727573361886785}, distributions={'max_depth': IntUniformDistribution(high=200, low=1), 'min_chiled_samples': IntUniformDistribution(high=500, low=5), 'min_chiled_weight': IntUniformDistribution(high=12, low=1), 'learning_rate': LogUniformDistribution(high=0.5, low=0.01), 'colsample_bytree': DiscreteUniformDistribution(high=1, low=0.7, q=0.1), 'sugsample': DiscreteUniformDistribution(high=1, low=0.7, q=0.1), 'lambda_l1': UniformDistribution(high=3.0, low=0.0), 'lambda_l2': UniformDistribution(high=3.0, low=0.0)}, user_attrs={}, system_attrs={'_number': 156}, intermediate_v